In [7]:
import numpy as np
import scipy.signal
from scipy.signal import find_peaks

########## 전처리 관련
def get_bcg_respiration_signal(bcg: np.ndarray, SR: int) -> np.ndarray:
    bcg = bcg.flatten()
    filter_coeffs = scipy.signal.butter(5, 0.7, 'low', fs=SR)
    return scipy.signal.filtfilt(*filter_coeffs, bcg)

def get_bcg_heartrate_signal(bcg: np.ndarray, SR: float) -> np.ndarray:
    bcg = bcg.flatten()
    filter_coeffs = scipy.signal.butter(5, [5, 15], 'band', fs=SR)
    return scipy.signal.filtfilt(*filter_coeffs, bcg)

def normalize_signal(signal: np.ndarray, window_size: int = 70) -> np.ndarray:
    normalized_signal = np.zeros_like(signal)
    
    for i in range(len(signal)):
        start_index = max(0, i - window_size + 1)
        window = signal[start_index:i+1]
        min_val = np.min(window)
        max_val = np.max(window)
        if max_val != min_val:
            normalized_signal[i] = (signal[i] - min_val) / (max_val - min_val)
        else:
            normalized_signal[i] = 0.5
    return normalized_signal

def calculate_checked_values(signal: np.ndarray, window_size: int = 10, threshold: float = 0.75):
    checked_values = np.zeros_like(signal)
    max_min_diff = np.zeros_like(signal)
    upto = np.zeros_like(signal)

    for i in range(len(signal)):
        if i >= window_size:
            window = signal[i-window_size+1:i+1]
            max_val = np.max(window)
            min_val = np.min(window)
            max_min_diff[i] = max_val - min_val
            if max_min_diff[i] >= threshold:
                checked_values[i] = max_val
                if checked_values[i-1] == 0:
                    upto[i] = max_val

    return checked_values, max_min_diff, upto

def calculate_upto_result(upto: np.ndarray):
    peaks = np.where(upto > 0)[0]
    
    if len(peaks) == 0:
        return np.zeros_like(upto)

    intervals = np.diff(peaks)

    if len(intervals) == 0:
        half_avg_interval = 0
    else:
        half_avg_interval = np.mean(intervals) / 2

    result = np.zeros_like(upto)
    
    for i in range(len(peaks)):
        if i == 0:
            result[peaks[i]] = upto[peaks[i]]

        elif (i>0) and (intervals[i-1] >= half_avg_interval):
            result[peaks[i]] = upto[peaks[i]]            

    return result

def calculate_permin(result: np.ndarray, sr: int):
    peaks = np.where(result > 0)[0]
    intervals = np.diff(peaks) / sr
    bpm = 60.0 / np.mean(intervals)
    return bpm

def find_minima_and_calculate_rr(signal: np.ndarray, sampling_rate: int):
    minima, _ = find_peaks(-signal)
    num_inflection_points = len(minima)
    time_duration = len(signal) / sampling_rate
    breaths = num_inflection_points
    respiratory_rate = breaths / time_duration * 60
    
    return respiratory_rate, minima


def preprocess_data(time, bcg, sampling_rate=100, normwindow=70, checkwindow=10, checkthereshold=0.75, run_model=False):
    filtered_hr = get_bcg_heartrate_signal(bcg, sampling_rate)
    filtered_rp = get_bcg_respiration_signal(bcg, sampling_rate)
    print(filtered_hr)
    print(filtered_rp)
    normalized_signal_h = normalize_signal(filtered_hr, window_size=normwindow)
    normalized_signal_r = normalize_signal(filtered_rp, window_size=normwindow)
    
    checked_values_h, max_min_diff_h, upto_h = calculate_checked_values(normalized_signal_h, window_size=checkwindow, threshold=checkthereshold)
    peak_h = calculate_upto_result(upto_h)
    bpm_h = calculate_permin(peak_h, sampling_rate)
    
    bpm_r, minima_r = find_minima_and_calculate_rr(normalized_signal_r, sampling_rate)
    
    combined_matrix_for_s = np.column_stack((time, filtered_hr, filtered_rp))
    
    if run_model :
        combined_matrix_for_m = np.stack([normalized_signal_h, normalized_signal_r, peak_h], axis=-1)
        return bpm_h, bpm_r, combined_matrix_for_s, combined_matrix_for_m
    
    return bpm_h, bpm_r, combined_matrix_for_s, None

In [9]:
import numpy as np

# 입력 데이터 생성
time = np.arange(0, 80, 0.1)
bcg = np.random.uniform(8000, 8500, 800)  # 8000에서 8500 사이의 임의의 값으로 BCG 신호 데이터 생성

# 함수 호출
bpm_h, bpm_r, combined_matrix_for_s, combined_matrix_for_m = preprocess_data(time, bcg)

combined_matrix_for_s

[  -1.754612     -1.06880775   11.04582556   31.32002669   41.53283542
   23.48629444  -22.4865617   -72.09063433  -91.78015364  -63.85475183
   -2.04718156   56.22843713   76.43529443   52.04015896    8.42895013
  -17.42030156   -5.30724882   33.783025     67.31563583   65.90195212
   23.58661276  -40.23404756  -95.8418548  -121.71777001 -113.2421838
  -78.51019406  -28.67615694   27.55833917   82.0958824   123.10169498
  135.48896957  108.70551716   46.74593868  -28.3836079   -85.02080349
  -98.8303605   -67.9295571   -15.27153044   24.92284254   28.81961074
   -1.72325649  -41.1219264   -58.25771098  -38.04837431    8.88115591
   54.68967706   73.77388938   58.57908918   21.27038312  -17.50287505
  -42.11180385  -47.53038571  -35.68197089   -9.65658103   26.93612387
   65.58402965   90.36175927   83.34292809   36.30334092  -39.96825945
 -117.11089598 -162.13576419 -154.17650803  -94.8053448    -6.40316003
   79.29409905  135.36469149  149.47462462  125.71818318   79.34691814
   28.2

array([[ 0.00000000e+00, -1.75461200e+00,  8.12012010e+03],
       [ 1.00000000e-01, -1.06880775e+00,  8.12487704e+03],
       [ 2.00000000e-01,  1.10458256e+01,  8.12961802e+03],
       ...,
       [ 7.97000000e+01, -7.64706364e+01,  8.23924555e+03],
       [ 7.98000000e+01, -5.52251153e+01,  8.23924379e+03],
       [ 7.99000000e+01, -1.19671662e+00,  8.23924240e+03]])